# RSI_Hedge_v1.3

## Setup

### Packages

In [1]:
from utils.packages import *

### Inputs and Parameters

#### Read Yaml files

In [2]:
temp_file = 'config/access_keys.yaml'
with open(temp_file) as temp_file:
    config = yaml.load(temp_file)
    
temp_file = 'config/currencies.yaml'
with open(temp_file) as temp_file:
    currs = yaml.load(temp_file)        

#### Yaml Variables

In [3]:
data = {}
data['instrument'] = "EUR_USD"
data['pip_size'] = currs['currs'][data['instrument']]['pip_size']

access_token = config['oanda_demo_hedge']['token']
accountID = config['oanda_demo_hedge']['account_id']
params = {'instruments': data['instrument']}

In [4]:
api = API(access_token = access_token)
request_stream_data = pricing.PricingStream(accountID=accountID, params=params)
response_stream = api.request(request_stream_data)

In [5]:
data['limit_iters'] = True
data['iter_max'] = 180

data['rsi_len'] = 14
data['rsi_ob'] = 70
data['rsi_os'] = 30
data['max_rsi'] = 0
data['min_rsi'] = 100
data['rsi_status'] = None

data['order_val'] = 1
data['stop_loss'] = 20

data['stop_loss_pip'] = data['stop_loss'] * data['pip_size']


data['order_flag'] = False
data['order_type'] = None
data['order_count'] = 0
data['positions_info'] = None

data['tick_list'] = collections.deque([])
data['UpPrice'] = collections.deque([])
data['DownPrice'] = collections.deque([])
data['AvgGain'] = collections.deque([])
data['AvgLoss'] = collections.deque([])
data['RS_List'] = collections.deque([])
data['RSI_List'] = collections.deque([])

In [6]:
#--------------------------------------------------------------------------------------------------------------------------
def print_report(iter, print_data):
    global data
    display.clear_output(wait = True)
    print("-----------------------------------")
    print(f"Iteration : {iter}")
    for print_iter, print_val in enumerate(print_data):
        print(f"Data_{print_iter} : {print_val}")
    print("-----------------------------------")
    return()
#==========================================================================================================================


#--------------------------------------------------------------------------------------------------------------------------
# Get timestamp of the price and segregate it
def get_date_time(resp):
    global data
    time_stamp = resp['time']
    data['date_val'], full_time = time_stamp.split(sep = 'T')
    data['time_val'], data['time_fraction'] = full_time.split(sep = '.')
    return()
#==========================================================================================================================


#--------------------------------------------------------------------------------------------------------------------------
# Get bid and ask prices
def get_prices(resp):    
    global data
    data['bid_price'] = float(resp['bids'][0]['price'])    
    data['ask_price'] = float(resp['asks'][0]['price'])
    data['spread'] = data['ask_price'] - data['bid_price']
    data['tick_price'] = (data['ask_price'] + data['bid_price']) / 2
    return()
#==========================================================================================================================


#--------------------------------------------------------------------------------------------------------------------------
def before_rsi_len():
    global data
    data['tick_list'].append(data['tick_price'])

    if len(data['tick_list']) == 1:
        data['UpPrice'].append(0)
        data['DownPrice'].append(0)
        data['AvgGain'].append(0)
        data['AvgLoss'].append(0)
        data['RS_List'].append(0)
        data['RSI_List'].append(0)
    elif len(data['tick_list']) > 1:
        old_price = data['tick_list'][len(data['tick_list'])-2]
        new_price = data['tick_price']
        if new_price > old_price:
            data['UpPrice'].append(new_price - old_price)
            data['DownPrice'].append(0)
            data['AvgGain'].append(np.mean(data['UpPrice']))
            data['AvgLoss'].append(np.mean(data['DownPrice']))
            data['RS_List'].append(data['AvgGain'][len(data['AvgGain'])-1]/data['AvgLoss'][len(data['AvgLoss'])-1])
            data['RSI'] = 100 - (100/(1+data['RS_List'][len(data['RS_List'])-1]))
            data['RSI_List'].append(data['RSI'])            
        elif old_price > new_price:
            data['UpPrice'].append(0)
            data['DownPrice'].append(old_price - new_price)
            data['AvgGain'].append(np.mean(data['UpPrice']))
            data['AvgLoss'].append(np.mean(data['DownPrice']))
            data['RS_List'].append(data['AvgGain'][len(data['AvgGain'])-1]/data['AvgLoss'][len(data['AvgLoss'])-1])
            data['RSI'] = 100 - (100/(1+data['RS_List'][len(data['RS_List'])-1]))
            data['RSI_List'].append(data['RSI'])            
        elif old_price == new_price:
            data['UpPrice'].append(0)
            data['DownPrice'].append(0)
            data['AvgGain'].append(np.mean(data['UpPrice']))
            data['AvgLoss'].append(np.mean(data['DownPrice']))
            data['RS_List'].append(data['AvgGain'][len(data['AvgGain'])-1]/data['AvgLoss'][len(data['AvgLoss'])-1])
            data['RSI'] = 100 - (100/(1+data['RS_List'][len(data['RS_List'])-1]))
            data['RSI_List'].append(data['RSI'])            
    display.clear_output(wait = True)
    print(f"Building RSI Tick list : {len(data['tick_list'])}")
    return()
#==========================================================================================================================    


#--------------------------------------------------------------------------------------------------------------------------
def after_rsi_len():
    global data
    data['UpPrice'].popleft()
    data['DownPrice'].popleft()
    data['AvgGain'].popleft()
    data['AvgLoss'].popleft()
    data['RS_List'].popleft()
    data['RSI_List'].popleft()
    data['tick_list'].popleft()
    data['tick_list'].append(data['tick_price'])

    old_price = data['tick_list'][len(data['tick_list'])-2]
    new_price = data['tick_price']
    if new_price > old_price:
        data['UpPrice'].append(new_price - old_price)
        data['DownPrice'].append(0)
        data['AvgGain'].append(np.mean(data['UpPrice']))
        data['AvgLoss'].append(np.mean(data['DownPrice']))
        data['RS_List'].append(data['AvgGain'][len(data['AvgGain'])-1]/data['AvgLoss'][len(data['AvgLoss'])-1])
        data['RSI'] = 100 - (100/(1+data['RS_List'][len(data['RS_List'])-1]))
        data['RSI_List'].append(data['RSI'])
    elif old_price > new_price:
        data['UpPrice'].append(0)
        data['DownPrice'].append(old_price - new_price)
        data['AvgGain'].append(np.mean(data['UpPrice']))
        data['AvgLoss'].append(np.mean(data['DownPrice']))
        data['RS_List'].append(data['AvgGain'][len(data['AvgGain'])-1]/data['AvgLoss'][len(data['AvgLoss'])-1])
        data['RSI'] = 100 - (100/(1+data['RS_List'][len(data['RS_List'])-1]))
        data['RSI_List'].append(data['RSI'])
    elif old_price == new_price:
        data['UpPrice'].append(0)
        data['DownPrice'].append(0)
        data['AvgGain'].append(np.mean(data['UpPrice']))
        data['AvgLoss'].append(np.mean(data['DownPrice']))
        data['RS_List'].append(data['AvgGain'][len(data['AvgGain'])-1]/data['AvgLoss'][len(data['AvgLoss'])-1])
        data['RSI'] = 100 - (100/(1+data['RS_List'][len(data['RS_List'])-1]))
        data['RSI_List'].append(data['RSI'])            

    return()
#==========================================================================================================================    


#--------------------------------------------------------------------------------------------------------------------------
def check_for_open_orders():
    global data
    
    request_position_data = positions.OpenPositions(accountID=accountID)
    data['positions_info'] = api.request(request_position_data)

    if len(data['positions_info']['positions']) == 0:
        data['order_flag'] = False

    elif len(data['positions_info']['positions']) == 1:
        data['order_flag'] = True
        data['order_count'] = 1
        data['long_positions'] = abs(int(data['positions_info']['positions'][0]['long']['units']))
        data['short_positions'] = abs(int(data['positions_info']['positions'][0]['short']['units']))
        if data['long_positions'] >= 1 and data['short_positions'] == 0:
            data['order_type'] = 'long'
        if data['short_positions'] >= 1 and data['long_positions'] == 0:
            data['order_type'] = 'short'            
        
    elif len(data['positions_info']['positions']) == 2:
        data['order_flag'] = True
        data['order_count'] = 2

    elif len(data['positions_info']['positions']) > 2:
        data['order_flag'] = True
        data['order_count'] = 3

    return()
#==========================================================================================================================    

#--------------------------------------------------------------------------------------------------------------------------
def check_rsi_status():
    global data
    if not data['order_flag']:
        if data['RSI'] <= data['rsi_os']:
            data['rsi_status'] = 'OS'
        elif data['RSI'] >= data['rsi_ob']:
            data['rsi_status'] = 'OB'
        else:
            data['rsi_status'] = None
            data['order_type'] = None
    return()
#==========================================================================================================================

#--------------------------------------------------------------------------------------------------------------------------
def rsi_peak_check():
    global data
    if not data['order_flag']:
        if data['rsi_status'] == 'OB':
            data['max_rsi'] = max(data['max_rsi'], data['RSI'])
            if data['RSI'] < data['max_rsi']:
                data['order_type'] = 'short'
                data['max_rsi'] = 0
                
        if data['rsi_status'] == 'OS':
            data['min_rsi'] = min(data['min_rsi'], data['RSI'])
            if data['RSI'] > data['min_rsi']:
                data['order_type'] = 'long'
                data['min_rsi'] = 100                
#==========================================================================================================================

#--------------------------------------------------------------------------------------------------------------------------
def make_order():
    global data
    if not data['order_flag']:
        if data['order_type'] == 'long':            
            data['order_val'] = data['order_val'] * 1
            
            data['stop_price'] = data['ask_price'] - data['stop_loss_pip']
            stopLossOnFill = StopLossDetails(price=data['stop_price'])
                       
            ordr = MarketOrderRequest(instrument = data['instrument'], 
                                      units=data['order_val'], 
                                      stopLossOnFill=stopLossOnFill.data)

            order_request_data = orders.OrderCreate(accountID, data=ordr.data)
            
            data['order_response'] = api.request(order_request_data)
            data['order_flag'] = True
            return()

        if data['order_type'] == 'short':
            data['order_val'] = data['order_val'] * -1
            
            data['stop_price'] = data['bid_price'] + data['stop_loss_pip']
            stopLossOnFill = StopLossDetails(price=data['stop_price'])

            
            ordr = MarketOrderRequest(instrument = data['instrument'], 
                                      units=data['order_val'], 
                                      stopLossOnFill=stopLossOnFill.data)
            
            order_request_data = orders.OrderCreate(accountID, data=ordr.data)
            
            data['order_response'] = api.request(order_request_data)
            data['order_flag'] = True
            return()
#==========================================================================================================================
          
    
#--------------------------------------------------------------------------------------------------------------------------

#==========================================================================================================================

In [7]:
for i, resp in enumerate(response_stream): 
    if data['limit_iters'] and i >= data['iter_max']:
        break
    if resp['type'] == 'HEARTBEAT': # Heart beat response to keep the api connection alive (Avoid timeout)
        continue
    
    get_date_time(resp)
    get_prices(resp)
        
    if len(data['tick_list']) < data['rsi_len']:
        before_rsi_len()          
        continue
    elif len(data['tick_list']) == data['rsi_len']:
        after_rsi_len()        
        
    check_for_open_orders()
    check_rsi_status()
    rsi_peak_check()
    
    make_order()
    
    if data['order_flag']:
        if data['order_type'] == 'long':
    
    if data['order_flag']:
        print(data['order_response'])
        
    print_report(i, [data['RSI'],data['rsi_status'],data['order_type']])
    l = ['tick_list', 'UpPrice', 'DownPrice', 'AvgGain', 'AvgLoss' , 'RS_List', 'RSI_List']
    for i, val in enumerate(l):
        print(f'{val} : {len(data[val])}')
    display.clear_output(wait = True)

V20Error: {"orderRejectTransaction":{"id":"410","accountID":"101-003-15069707-003","userID":15069707,"batchID":"410","requestID":"24747030953485778","time":"2020-11-12T12:37:40.417275715Z","type":"MARKET_ORDER_REJECT","rejectReason":"TRAILING_STOP_LOSS_ON_FILL_PRICE_DISTANCE_MINIMUM_NOT_MET","instrument":"EUR_USD","units":"1","timeInForce":"FOK","positionFill":"DEFAULT","stopLossOnFill":{"price":"1.17860","timeInForce":"GTC"},"trailingStopLossOnFill":{"distance":"0.00010","timeInForce":"GTC"},"reason":"CLIENT_ORDER"},"relatedTransactionIDs":["410"],"lastTransactionID":"410","errorMessage":"The Trailing Stop Loss on fill price distance does not meet the minimum allowed amount","errorCode":"TRAILING_STOP_LOSS_ON_FILL_PRICE_DISTANCE_MINIMUM_NOT_MET"}

In [ ]:
data['positions_info']['positions'][0]

In [ ]:
abs(int(data['positions_info']['positions'][0]['short']['units']))

In [ ]:
data['positions_info']['positions'][0].keys()

In [ ]:
data['stop_price'] = data['ask_price'] - data['stop_loss_pip']
stopLossOnFill = StopLossDetails(price=data['stop_price'])
data['order_val'] = data['order_val'] * 1

ordr = MarketOrderRequest(instrument = data['instrument'], units=data['order_val'], stopLossOnFill=stopLossOnFill.data)
order_request_data = orders.OrderCreate(accountID, data=ordr.data)

data['order_response'] = api.request(order_request_data)

In [ ]:
sld = 0.0001
trailingStopLossOnFill = TrailingStopLossDetails(distance=sld)

In [ ]:

print(trailingStopLossOnFill)
ordr = MarketOrderRequest(
    instrument="EUR_USD",
    units=10000,
    trailingStopLossOnFill=trailingStopLossOnFill.data
)
print(json.dumps(ordr.data, indent=4))
r = orders.OrderCreate(accountID, data=ordr.data)
rv = client.request(r)